In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField


In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("Kafka to MinIO Pipeline")
         .getOrCreate())

25/02/11 13:03:28 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.27 instead (on interface en0)
25/02/11 13:03:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Library/Caches/pypoetry/virtualenvs/realtime-fraud-detection-system-eUiKKk2v-py3.13/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6a366728-ee1e-483f-b13c-d168cf0d2856;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [4]:
def execute_query(query: str):
    spark.sql(query).show(truncate=False)

In [5]:
# Kafka configurations
kafka_brokers = "localhost:9093"  # Update with your Kafka broker address
kafka_topic = "debezium.payment.transactions"
# Read data from Kafka (batch mode)
kafka_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_brokers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

In [30]:
kafka_df.select("value").show(truncate=False)

25/02/11 12:34:18 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/11 12:34:18 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/11 12:34:18 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/11 12:34:18 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/11 12:34:18 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
kafka_df.select("value.payload").printSchema()


AnalysisException: [INVALID_EXTRACT_BASE_FIELD_TYPE] Can't extract a value from "value". Need a complex type [STRUCT, ARRAY, MAP] but got "BINARY".

In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, LongType

# Define the schema for the JSON messages
root_schema = StructType([
    StructField("schema", StructType([
        StructField("type", StringType(), False),
        StructField("fields", StringType(), True),  # Simplified for nested schema
        StructField("optional", StringType(), True),
        StructField("name", StringType(), True),
        StructField("version", IntegerType(), True)
    ]), True),
    StructField("payload", StructType([
        StructField("before", StructType([
            StructField("tx_id", IntegerType(), True),
            StructField("tx_datetime", LongType(), True),
            StructField("customer_id", IntegerType(), True),
            StructField("terminal_id", IntegerType(), True),
            StructField("tx_amount", StringType(), True)  # Use StringType to handle Base64-encoded Decimal
        ]), True),
        StructField("after", StructType([
            StructField("tx_id", IntegerType(), True),
            StructField("tx_datetime", LongType(), True),
            StructField("customer_id", IntegerType(), True),
            StructField("terminal_id", IntegerType(), True),
            StructField("tx_amount", StringType(), True) 
        ]), True),
        StructField("source", StructType([
            StructField("version", StringType(), True),
            StructField("connector", StringType(), True),
            StructField("name", StringType(), True),
            StructField("ts_ms", LongType(), True),
            StructField("snapshot", StringType(), True),
            StructField("db", StringType(), True),
            StructField("sequence", StringType(), True),
            StructField("ts_us", LongType(), True),
            StructField("ts_ns", LongType(), True),
            StructField("schema", StringType(), True),
            StructField("table", StringType(), True),
            StructField("txId", LongType(), True),
            StructField("lsn", LongType(), True),
            StructField("xmin", LongType(), True)
        ]), True),
        StructField("transaction", StructType([
            StructField("id", StringType(), True),
            StructField("total_order", LongType(), True),
            StructField("data_collection_order", LongType(), True)
        ]), True),
        StructField("op", StringType(), True),
        StructField("ts_ms", LongType(), True),
        StructField("ts_us", LongType(), True),
        StructField("ts_ns", LongType(), True)
    ]))
])

from pyspark.sql.functions import from_json
kafka_df = kafka_df.withColumn("value", col("value").cast(StringType())) \
    .withColumn("key", col("key").cast(StringType()))
# Apply the schema to the DataFrame
kafka_df = kafka_df.withColumn("value", from_json("value", root_schema))
kafka_df.createOrReplaceTempView("kafkadata")






25/02/11 13:03:53 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [8]:
execute_query("select * from kafkadata")

25/02/11 13:04:01 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/11 13:04:01 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/11 13:04:01 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/11 13:04:01 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/11 13:04:01 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
execute_query(""" create or replace temp view latest_data as 
              with extracted_data  as (
                    select value.payload.after.tx_id,
                    value.payload.after.tx_datetime, 
                    value.payload.after.customer_id,
                    value.payload.after.terminal_id,
                    value.payload.after.tx_amount,
                    timestamp
                    from kafkadata),
              ranked_data as (
                  select tx_id,
                tx_datetime,
                customer_id,
                terminal_id,
                tx_amount,
                  timestamp,
                  ROW_NUMBER() over (partition by tx_id order by timestamp DESC) as rn
                  from extracted_data) 
                select tx_id,
                    tx_datetime,
                    customer_id,
                    terminal_id,
                    tx_amount,
                  now() as row_created_timestamp,
                  now() as row_updated_timestamp
              from ranked_data 
              where rn=1""")

++
||
++
++



In [10]:
execute_query("CREATE DATABASE IF NOT EXISTS payment;")
execute_query("show databases")

++
||
++
++

+---------+
|namespace|
+---------+
|default  |
|payment  |
+---------+



In [11]:
execute_query("""
      CREATE TABLE IF NOT EXISTS payment.transactions (
                    tx_id INT,
                    tx_datetime Timestamp,
                    customer_id INT,
                    terminal_id INT,
                    tx_amount DECIMAL(10,2),
    row_created_timestamp TIMESTAMP,
    row_updated_timestamp TIMESTAMP
)
USING iceberg
""")

++
||
++
++



In [12]:
execute_query("select * from payment.transactions")

+-----+-----------+-----------+-----------+---------+---------------------+---------------------+
|tx_id|tx_datetime|customer_id|terminal_id|tx_amount|row_created_timestamp|row_updated_timestamp|
+-----+-----------+-----------+-----------+---------+---------------------+---------------------+
+-----+-----------+-----------+-----------+---------+---------------------+---------------------+

